In [1]:
import torch
import torch.nn.functional as F
import sympy as sp
import pandas as pd
import numpy as np
import numdifftools as nd

In [2]:
device = torch.device("cuda:4" if torch.cuda.is_available() else "cpu")

In [3]:
df = pd.read_csv('physics_equations.csv')

functions = []
num_vars_per_func = []

for _, row in df.iterrows():
    formula = row['Formula']
    num_vars = row['# variables']
    function_details = {
        'formula': formula,
        'variables': []
    }
    
    for i in range(1, 11):  
        v_name = row.get(f'v{i}_name', None)
        v_low = row.get(f'v{i}_low', None)
        v_high = row.get(f'v{i}_high', None)
        
        if pd.notna(v_name):
            function_details['variables'].append({
                'name': v_name,
                'low': v_low,
                'high': v_high
            })
    
    functions.append(function_details)
    num_vars_per_func.append(num_vars)

'''for i, func in enumerate(functions):
    print(f"Function {i+1}:")
    print(f"  Formula: {func['formula']}")
    print(f"  Number of Variables: {num_vars_per_func[i]}")
    print("  Variables:")
    for var in func['variables']:
        print(f"    - Name: {var['name']}, Range: ({var['low']}, {var['high']})")
    print()'''

'for i, func in enumerate(functions):\n    print(f"Function {i+1}:")\n    print(f"  Formula: {func[\'formula\']}")\n    print(f"  Number of Variables: {num_vars_per_func[i]}")\n    print("  Variables:")\n    for var in func[\'variables\']:\n        print(f"    - Name: {var[\'name\']}, Range: ({var[\'low\']}, {var[\'high\']})")\n    print()'

In [4]:
def generate_function(function, sample_size, x, max_vars, device):
    sympy_symbols = []
    param_tensors = []
    
    for var in function["variables"]:
        sym = sp.symbols(var["name"])
        sympy_symbols.append(sym)
        min_val, max_val = var["low"], var["high"]
        param = (max_val - min_val) * torch.rand(sample_size, 1, device=device) + min_val
        param_tensors.append(param)

    sympy_symbols.append(sp.symbols('x'))
    params = torch.cat(param_tensors, dim=1)
    padded_params = F.pad(params, pad=(0, max_vars - params.size(1)))
    padded_params = padded_params.expand(sample_size, max_vars)
    formula = sp.sympify(function["formula"])
    eval_func = sp.lambdify(sympy_symbols, formula, modules="numpy")
        
    results = []
    for xi in x:
        input_values = torch.cat([params, xi.expand(sample_size, 1)], dim=1)
        results.append(eval_func(*input_values.T))
    results = torch.stack(results, dim=1)
    
    return results, formula, sympy_symbols, padded_params

In [5]:
sample_size = 1000
sequence_length = 100
num_funcs = 10
max_vars = 5

x_values = torch.linspace(-1, 1, sequence_length).to(device)
hold = []
for f in functions[0:10]:
    try:
        results = generate_function(f, sample_size, x_values, max_vars, device)
        hold.append(results)
    except Exception as e:
        print(f"Error processing function {f}: {e}")

In [6]:
y_values = torch.stack([l[0] for l in hold])
formulas = [l[1] for l in hold]
symbols = [l[2] for l in hold]
param_values = torch.stack([l[3] for l in hold])
num_params = torch.tensor([len(l[2]) for l in hold])

In [7]:
def evaluate_function(params, symbols, formula, x):
    var_values = {symbols[j]: params[:, j] for j in range(len(symbols)-1)}
    eval_func = sp.lambdify(symbols, formula, modules="numpy")
    results = []
    for xi in x:
        var_values[symbols[-1]] = xi
        np_values = {str(sym): var_values[sym].detach().cpu().numpy() for sym in symbols}
        results.append(eval_func(**np_values))
    tensor_results = [torch.tensor(r, device=device) for r in results]
    return torch.stack(tensor_results, dim=1)

In [8]:
epsilon = 1e-4
derivatives = torch.zeros(num_funcs, sample_size, sequence_length, max_vars)
for f in range(num_funcs):
    params_n = param_values[f].clone().detach().requires_grad_(True)
    for p in range(len(symbols[f])):
        plus = params_n.clone()
        minus = params_n.clone()
        plus[:,p] += epsilon
        forward_values = evaluate_function(plus, symbols[f], formulas[f], x_values)
        minus[:, p] -= epsilon
        backward_values = evaluate_function(minus, symbols[f], formulas[f], x_values)
        derivatives[f, :, :, p] = (forward_values - backward_values) / (2 * epsilon)
derivatives.shape

torch.Size([10, 1000, 100, 5])

In [18]:
hessians = torch.zeros(num_funcs, sample_size, sequence_length, max_vars, max_vars)
for f in range(num_funcs):
    params_f = param_values[f].clone().detach().requires_grad_(True)
    for j in range(len(symbols[f])):
        for k in range(len(symbols[f])):
            plus_plus = params_f.clone()
            plus_minus = params_f.clone()
            minus_plus = params_f.clone()
            minus_minus = params_f.clone()

            plus_plus[:, j] += epsilon
            plus_plus[:, k] += epsilon

            plus_minus[:, j] += epsilon
            plus_minus[:, k] -= epsilon

            minus_plus[:, j] -= epsilon
            minus_plus[:, k] += epsilon

            minus_minus[:, j] -= epsilon
            minus_minus[:, k] -= epsilon

            forward_forward = evaluate_function(plus_plus, symbols[f], formulas[f], x_values)
            forward_backward = evaluate_function(plus_minus, symbols[f], formulas[f], x_values)
            backward_forward = evaluate_function(minus_plus, symbols[f], formulas[f], x_values)
            backward_backward = evaluate_function(minus_minus, symbols[f], formulas[f], x_values)
            hessians[f, :, :, j, k] = (forward_forward - forward_backward - backward_forward + backward_backward) / (4 * epsilon **2)
hessians.shape

torch.Size([10, 1000, 100, 5, 5])

(tensor([[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]),
 tensor(-1., device='cuda:4'),
 t/x,
 tensor([0.1069, 0.0000, 0.0000, 0.0000, 0.0000], device='cuda:4'))

In [42]:
flat_result = torch.flatten(torch.stack(y_values).unsqueeze(2), start_dim=0, end_dim=1)

max_der = max(d.size(1) for d in derivatives)
padded_ders = []
for d in derivatives:
    padded_tensor = F.pad(d, pad=(0, max_der - d.size(1)))
    padded_ders.append(padded_tensor)
flat_der = torch.flatten(torch.stack(padded_ders), start_dim=0, end_dim=1)

max_hess = max_der**2
padded_hess = []
for h in hessians:
    h = torch.flatten(h, start_dim=1)
    padded_tensor = F.pad(h, pad=(0, max_hess - h.size(1)))
    padded_hess.append(padded_tensor)
flat_hess = torch.flatten(torch.stack(padded_hess), start_dim=0, end_dim=1)

flat_der = F.pad(flat_der, (0, flat_hess.size(-1) - flat_der.size(-1)))
flat_result = F.pad(flat_result, (0, flat_hess.size(-1) - flat_result.size(-1)))

flat_result.shape, flat_der.shape, flat_hess.shape

(torch.Size([10000, 81]), torch.Size([10000, 81]), torch.Size([10000, 81]))

In [ ]:

num_params = torch.repeat_interleave(num_params, repeats=sample_size)
y_values = torch.flatten(y_values, end_dim=1)
param_values = torch.flatten(param_values, end_dim=1)
formulas = np.array([[formula for _ in range(1000)] for formula in formulas]).flatten()
symbols = []
for s in symbols_hold:
    for _ in range(sample_size):
        symbols.append(s)

y_values.shape, param_values.shape, num_params.shape, formulas.shape, len(symbols)

In [44]:
torch.save({
    'flattened_data': torch.stack([flat_result, flat_der, flat_hess], dim=2),
    'results': y_values,
    'formulas': hold_formulas,
    'symbols': hold_symbols,
    'params': param_values,
    'num_params': num_params,
}, 'hold_data.pth')